In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames",
    "uCSV",
    "Kmers",
    "CodecZlib",
    "StatsPlots",
    "GLM",
    "StatsBase",
    "CSV"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
refseq_metadata = Mycelia.load_refseq_metadata();

In [ ]:
# function plot_kmer_frequency_spectra_with_trendline(counts; log_scale = log2, kwargs...)
#     kmer_counts_hist = StatsBase.countmap(c for c in counts)
#     xs = collect(keys(kmer_counts_hist))
#     ys = collect(values(kmer_counts_hist))
#     if isa(log_scale, Function)
#         xs = log_scale.(xs)
#         ys = log_scale.(ys)
#     end
    
#     p = StatsPlots.plot(
#         xs,
#         ys,
#         xlims = (0, maximum(xs) + max(1, ceil(0.1 * maximum(xs)))),
#         ylims = (0, maximum(ys) + max(1, ceil(0.1 * maximum(ys)))),
#         seriestype = :scatter,
#         legend = false,
#         xlabel = isa(log_scale, Function) ? "$(log_scale)(observed frequency)" : "observed frequency",
#         ylabel = isa(log_scale, Function) ? "$(log_scale)(# of kmers)" : "observed frequency",
#         ;kwargs...
#     )
#     return p
# end

# function plot_kmer_frequency_spectra(counts; log_scale = log2, kwargs...)
#     kmer_counts_hist = StatsBase.countmap(c for c in counts)
#     xs = collect(keys(kmer_counts_hist))
#     ys = collect(values(kmer_counts_hist))
#     if isa(log_scale, Function)
#         xs = log_scale.(xs)
#         ys = log_scale.(ys)
#     end
    
#     p = StatsPlots.plot(
#         xs,
#         ys,
#         xlims = (0, maximum(xs) + max(1, ceil(0.1 * maximum(xs)))),
#         ylims = (0, maximum(ys) + max(1, ceil(0.1 * maximum(ys)))),
#         seriestype = :scatter,
#         legend = false,
#         xlabel = isa(log_scale, Function) ? "$(log_scale)(observed frequency)" : "observed frequency",
#         ylabel = isa(log_scale, Function) ? "$(log_scale)(# of kmers)" : "observed frequency",
#         ;kwargs...
#     )
#     return p
# end

In [ ]:
# Danio rerio
# 7955
# GCF_000002035.6

# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   2.059938 seconds (617 allocations: 309.625 KiB)
#   0.000201 seconds (116 allocations: 9.531 KiB)
# [ Info: /global/cfs/cdirs/m4269/cjprybol/Mycelia/projects/variant-calling-benchmarking/data/genomes/GCF_000002035.6/GCF_000002035.6_GRCz11_genomic.fna.gz.k7.canonical.jf.count_histogram.tsv already exists
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   2.051873 seconds (617 allocations: 309.641 KiB)
#   1.875662 seconds (181.84 k allocations: 11.496 MiB)
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   1.962617 seconds (613 allocations: 309.562 KiB)
#  21.618736 seconds (116.74 k allocations: 9.253 MiB)
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   1.977223 seconds (734 allocations: 320.039 KiB)
# 479.764688 seconds (100.25 k allocations: 8.412 MiB)

accession = "GCF_000002035.6"
outdir=mkpath(joinpath(genome_dir, accession))
ftp = first(refseq_metadata[(refseq_metadata[!, "refseq_category"] .== "reference genome") .& map(x -> x == accession, refseq_metadata[!, "#assembly_accession"]), "ftp_path"])
fasta = Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
kmer_counts_dict = Dict()
for k in [7, 11, 13, 17]
    @time jellyfish_counts_file = Mycelia.jellyfish_count(fastx=fasta, k = k, canonical=true)
    @time count_histogram_file = Mycelia.jellyfish_counts_to_kmer_frequency_histogram(jellyfish_counts_file)
    kmer_counts_dict[k] = CSV.read(count_histogram_file, DataFrames.DataFrame, delim='\t')
end
kmer_counts_dict

In [ ]:
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   2.707734 seconds (632 allocations: 310.141 KiB)
#   0.462371 seconds (101.25 k allocations: 8.673 MiB)
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   2.008665 seconds (615 allocations: 309.594 KiB)
#   2.110626 seconds (229.84 k allocations: 13.199 MiB)
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   2.025346 seconds (615 allocations: 309.594 KiB)
#  21.925132 seconds (175.54 k allocations: 11.459 MiB)
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   2.024590 seconds (615 allocations: 309.594 KiB)
# 832.700463 seconds (170.55 k allocations: 11.121 MiB)

# Mus musculus
# 10090
# GCF_000001635.27
accession = "GCF_000001635.27"
outdir=mkpath(joinpath(genome_dir, accession))
ftp = first(refseq_metadata[(refseq_metadata[!, "refseq_category"] .== "reference genome") .& map(x -> x == accession, refseq_metadata[!, "#assembly_accession"]), "ftp_path"])
fasta = Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
kmer_counts_dict = Dict()
for k in [7, 11, 13, 17]
    @time jellyfish_counts_file = Mycelia.jellyfish_count(fastx=fasta, k = k, canonical=true)
    @time count_histogram_file = Mycelia.jellyfish_counts_to_kmer_frequency_histogram(jellyfish_counts_file)
    kmer_counts_dict[k] = CSV.read(count_histogram_file, DataFrames.DataFrame, delim='\t')
end
kmer_counts_dict

In [ ]:
# Homo sapiens
# 9606
# GCF_000001405.40

# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   2.123828 seconds (620 allocations: 309.953 KiB)
#   0.425395 seconds (101.41 k allocations: 8.880 MiB)
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   1.656025 seconds (606 allocations: 309.172 KiB)
#   1.583714 seconds (251.21 k allocations: 15.355 MiB)
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   1.745315 seconds (609 allocations: 309.500 KiB)
#  20.587222 seconds (181.83 k allocations: 11.732 MiB)
# [ Info: conda environment kmer-jellyfish already present; set force=true to update/re-install
#   1.816905 seconds (608 allocations: 309.203 KiB)
# 991.531925 seconds (176.60 k allocations: 11.345 MiB)

accession = "GCF_000001405.40"
outdir=mkpath(joinpath(genome_dir, accession))
ftp = first(refseq_metadata[(refseq_metadata[!, "refseq_category"] .== "reference genome") .& map(x -> x == accession, refseq_metadata[!, "#assembly_accession"]), "ftp_path"])
fasta = Mycelia.download_genome_by_ftp(ftp=ftp, outdir=outdir)
kmer_counts_dict = Dict()
for k in [7, 11, 13, 17]
    @time jellyfish_counts_file = Mycelia.jellyfish_count(fastx=fasta, k = k, canonical=true)
    @time count_histogram_file = Mycelia.jellyfish_counts_to_kmer_frequency_histogram(jellyfish_counts_file)
    kmer_counts_dict[k] = CSV.read(count_histogram_file, DataFrames.DataFrame, delim='\t')
end
kmer_counts_dict

In [ ]:

# # Add back R-squared analysis
# # write out as PDF & PNG
# # counts = canonical_kmer_counts[!, "count"]
# kmer_counts_hist = StatsBase.countmap(counts)
# xs = log2.(collect(keys(kmer_counts_hist)))
# ys = log2.(collect(values(kmer_counts_hist)))
# # Fit the model
# model = GLM.lm(GLM.@formula(_ys ~ _xs), DataFrames.DataFrame(_xs=xs, _ys=ys))
# r2_value = GLM.r2(model)
# p = Mycelia.plot_kmer_frequency_spectra(counts)
# StatsPlots.plot!(p, title = basename(jellyfish_counts_file))
# StatsPlots.plot!(p, xs, GLM.predict(model))
# StatsPlots.annotate!(p, 0.5, 0.5, StatsPlots.text("r = $(round(r2_value, digits=2))", 10, :left))

# # Create sample data
# x = 0:0.1:10
# y1 = sin.(x)
# y2 = cos.(x)
# y3 = tan.(x)

# # Create a multi-panel plot
# p = StatsPlots.plot(layout = (3, 1), size = (600, 800), xlabel = "Shared X-axis", title = accession)

# # First subplot
# StatsPlots.plot!(p, x, y1, subplot = 1, ylabel = "Y1", title = "k=7", xlabel="")

# StatsPlots.plot!(p, x, y2, subplot = 2, ylabel = "Y2", title = "k=11", xlabel="")

# StatsPlots.plot!(p, x, y3, subplot = 2, ylabel = "Y2", title = "k=13", xlabel="")

# StatsPlots.plot!(p, x, y4, subplot = 2, ylabel = "Y2", title = "k=17", xlabel="")

# p